In [32]:
import numpy as np
import json
from utils import *

item_matrix = np.load('item_matrix.npy', allow_pickle=True )
item_id = np.load('itemid.npy', allow_pickle=True)
wts = np.load('weights.npy', allow_pickle=True)

f = open('anime_id2name.json')

id2name = json.load(f)
f.close

<function TextIOWrapper.close()>

In [3]:
from load import id2name

In [4]:
from thefuzz import fuzz
from thefuzz import process

In [5]:
names = id2name.values()

In [9]:
id2name

{'11013': 'Inu X Boku Secret Service',
 '2104': 'My Bride is a Mermaid',
 '5262': 'Shugo Chara!! Doki',
 '721': 'Princess Tutu',
 '12365': 'Bakuman.',
 '6586': 'Yume-iro Pâtissière',
 '178': 'Ultramaniac - Magical Girl',
 '2787': 'Shakugan no Shana: Season II',
 '4477': 'Nodame Cantabile: Paris-hen',
 '853': 'Ouran High School Host Club',
 '4814': 'Junjo Romantica 2',
 '7054': 'Maid Sama!',
 '11123': 'Sekai Ichi Hatsukoi - World&#039;s Greatest First Love 2',
 '14227': 'My Little Monster',
 '269': 'Bleach',
 '59': 'Chobits',
 '6045': 'Kimi ni Todoke: From Me to You',
 '1735': 'Naruto: Shippuden',
 '210': 'Ranma ½',
 '4224': 'Toradora!',
 '10030': 'Bakuman.',
 '74': 'Gakuen Alice',
 '4722': 'Skip Beat!',
 '14397': 'Chihayafuru 2',
 '1557': 'Shonen Onmyoji',
 '10800': 'Chihayafuru',
 '3731': 'ItaKiss',
 '9513': 'Beelzebub',
 '5835': 'Hanasakeru Seishounen',
 '9863': 'SKET Dance',
 '7817': 'Yamada&#039;s First Time: B Gata H Kei',
 '966': 'Shin Chan',
 '120': 'Fruits Basket',
 '957': 'The

In [11]:
process.extract("Attack on Titan", names, limit=1)[0][0]

'Attack on Titan'

In [2]:
import numpy as np
item_id = np.load('data/itemid.npy', allow_pickle=True)

In [3]:
item_id

array([    1,     5,     6, ..., 36825, 36848, 36861])

In [1]:
import numpy 
from load import item_factors, item_fea

item_factors.shape

(6668, 50)

In [16]:
from search import * 
search_res = search('god')
search_res

6015

In [18]:
id2name['6015']

'Detective Goddess'

In [20]:
id2name[str(item_id[4418])]

'Detective Goddess'

In [33]:
%%time
M = item_matrix @ item_matrix.T

CPU times: user 4.14 s, sys: 1.54 s, total: 5.68 s
Wall time: 1.73 s


In [34]:
import telebot
from telebot import types

ratings = {}
request_user = {}
API_TOKEN = '5677096457:AAHYpRxfaCw9WnJjRybbF-igrxK3XAe3cXo'
bot = telebot.TeleBot(API_TOKEN)

test_ids = [20, 21, 269, 136]


@bot.message_handler(commands=['suggest'])
def echo_message(message):
    name = message.from_user.username
    if name not in ratings:
        bot.send_message(message.chat.id,
                             text=f'Not enough info')
    else:
        p = np.zeros(13686)
        #print(ratings[name])
        for anime_id, score in ratings[name].items():
            p[anime_id] = score
        recs = M @ p
        recs = downvote_seen_items(recs, test_ids)
        for id_ in topn_recommendations(recs):
            bot.send_message(message.chat.id,
                             text=f'Consider {id2name[str(item_id[id_])]}')
        
    
                
@bot.message_handler(commands=['rate'])
def echo_message(message):
    name = message.from_user.username
    if name not in ratings:
        ratings[name] = {}

    keyboard = types.InlineKeyboardMarkup()
    keys = []
    for i in range(1, 11):
        keys.append(types.InlineKeyboardButton(text=f'{i}', callback_data=i))
    keyboard.add(*keys)
    
    anime_id = np.random.randint(13686)
    request_user[name] = anime_id

    bot.send_message(
        message.from_user.id, 
        text=f'Please rate {id2name[str(item_id[anime_id])]}', 
        reply_markup=keyboard
        )
    

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    name = call.from_user.username
    ratings[name][request_user[name]] = call.data
    bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.id,
                           text=f'{id2name[str(item_id[request_user[name]])]}: {call.data}')
    
        

    
@bot.message_handler(content_types=['text'])
def echo_message(message):
    name = message.from_user.username 
    if name in request_user:
        ratings[name][request_user[name]] = float(message.text)      
                
    

In [35]:
bot.polling()